# Pizza pizza pizza

Get top venues by keyword and location, querying Google, Yelp, Foursquare

#### Motivation:
 - Gmaps and Yelp GUIs sorting and filtering are limited.
 - Query all 3 services and bring combined results into sortable table and map widget

#### Google

 - Needs a Google API key and module
 - [Create Google Cloud credentials and give access to Places APIs](https://console.cloud.google.com/google/maps-apis/credentials) (also, restrict IP or set other restrictions)
 - `conda install -c conda-forge -y gmaps`
 - put key in `apikey.txt`
 - `gmaps` Jupyter nbextension to show maps in notebook, with marker pins etc.

```
conda install -c conda-forge -y jupyter_contrib_nbextensions
jupyter nbextension enable --py gmaps
jupyter notebook
```

#### Yelp
 - needs Yelp API key and module
 - https://www.yelp.com/developers/documentation/v3
 - https://github.com/gfairchild/yelpapi
 - put key in `yelpkey.txt`
 
#### Foursquare
- Needs Foursquare API key and module
- https://developer.foursquare.com/docs/places-api/getting-started/
- https://github.com/mLewisLogic/foursquare
- OAuth id in `foursquare_id.txt`
- OAuth secret in `foursquare_secret.txt`

See `requirements.txt` for versions used, other requirements (requests, beakerx, folium, Flask)

Was going to try OpenTable and TripAdvisor but their language seems to limit API key access to approved commercial partners.
- https://dev.opentable.com/affiliate-partners/
- https://www.tripadvisor.com/APIAccessSupport

In [36]:
import time
from pprint import pprint
import ipywidgets
from ipywidgets import widgets, interact
from itertools import product
from os import path

import traceback
import pdb

import numpy as np
import pandas as pd

# cluster similar entities
import pandas_dedupe

import sklearn
from sklearn.preprocessing import StandardScaler

import requests, json 

import qgrid
from qgrid import show_grid

# convert coords to km using haversine distance 
import geopy
from geopy.distance import distance

import gmaps
with open('apikeys/apikey.txt') as f:
    api_key = f.readline().strip()
    f.close
gmaps.configure(api_key=api_key)

# https://github.com/gfairchild/yelpapi
import yelpapi
from yelpapi import YelpAPI
with open('apikeys/yelpkey.txt') as f:
    yelp_key = f.readline().strip()
    f.close
yelp_api = YelpAPI(yelp_key)

import foursquare
from foursquare import Foursquare, FoursquareException
with open('apikeys/foursquare_id.txt') as f:
    foursquare_id = f.readline().strip()
    f.close
with open('apikeys/foursquare_secret.txt') as f:
    foursquare_secret = f.readline().strip()
    f.close

# interactive maps
import folium

print("%-20s %s" % ("numpy", np.__version__))
print("%-20s %s" % ("pandas", pd.__version__))
print("%-20s %s" % ("ipywidgets", ipywidgets.__version__))
print("%-20s %s" % ("qgrid", qgrid.__version__))
print("%-20s %s" % ("sklearn", sklearn.__version__))
print("%-20s %s" % ("requests", requests.__version__))
print("%-20s %s" % ("geopy", geopy.__version__))
print("%-20s %s" % ("gmaps", gmaps.__version__))
print("%-20s %s" % ("foursquare", foursquare.__version__))
print("%-20s %s" % ("folium", folium.__version__))


numpy                1.18.1
pandas               1.0.3
ipywidgets           7.5.1
qgrid                1.3.1
sklearn              0.22.1
requests             2.23.0
geopy                2.0.0
gmaps                0.9.0
foursquare           1!2020.1.30
folium               0.11.0


## Google Maps

In [37]:
# pick a search term
keyword_options = [('Pizza', 'pizza'), ('Coffee', 'coffee'), ('Ice Cream', 'icecream')]
keyword = 'pizza'

@interact
def get_kw(kw = widgets.Dropdown(
    options=keyword_options,
    value=keyword,
    description='Search term:  ',
    disabled=False,
)):
    global keyword
    keyword = kw
    return None


interactive(children=(Dropdown(description='Search term:  ', options=(('Pizza', 'pizza'), ('Coffee', 'coffee')…

In [3]:
# pick a location
location = '40.7484, -73.9857'
location_coords = tuple(eval(location))
location_options = [('Midtown', '40.7484, -73.9857'),
                    ('Downtown', '40.7077443,-74.0139089'),
                    ('Upper East Side', '40.7711473,-73.9661166'),
                    ('Upper West Side', '40.778794,-73.984257'),
                    ('Brooklyn Heights', '40.6915812,-73.9954095'), 
                    ('Grand Army Plaza', '40.671872,-73.972544'),
                    ('Bay Ridge', '40.6292633,-74.0309554'),
                    ('Williamsburg', '40.7144609,-73.9553373'),
                  ]

@interact
def get_loc(loc = widgets.Dropdown(
    options=location_options,
    value=location,
    description='Location:',
)):
    global location
    global location_coords
    location = loc
    location_coords = tuple(eval(loc))
    return None

interactive(children=(Dropdown(description='Location:', options=(('Midtown', '40.7484, -73.9857'), ('Downtown'…

In [4]:
# pick anything we don't have a pickle file for
for test_keyword, test_location in product(keyword_options, location_options):
    keyword = test_keyword[1]
    location_name = test_location[0]
    location = test_location[1]
    location_coords = tuple(eval(test_location[1]))
    test_pickle_filename = keyword + "_" + location_name.replace(' ', '').lower() + ".pkl"
    
    if not path.exists(test_pickle_filename):
        break
        
print(location_name, keyword, location, location_coords)
print(test_pickle_filename)

Williamsburg icecream 40.7144609,-73.9553373 (40.7144609, -73.9553373)
icecream_williamsburg.pkl


In [70]:
keyword = 'lunch'

In [71]:
# gmap of chosen location
figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}
fig = gmaps.figure(center=location_coords, zoom_level=12, layout=figure_layout)
fig.add_layer(gmaps.marker_layer([location_coords]))
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [72]:
# global options for all search services APIs
MIN_USER_RATINGS = 20
MIN_RATING = 0
NRESULTS = 50
RADIUS = 1000


In [73]:
# gmaps options
GMAPS_URL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# https://developers.google.com/places/web-service/search#TextSearchRequests
#https://developers.google.com/places/web-service/supported_types
# rankby='prominence'
rankby='distance'
ltype='establishment'


In [74]:
def gmaps_get_first_page(api_key, location, **kwargs):
    """get first page of results from gmaps using api_key, location, kwargs for search spec"""
    # use either rankby or radius kwarg
    request_url = GMAPS_URL + '?key=' + api_key
    request_url += '&location=' + location
    for name, val in kwargs.items():
        request_url += '&' + name + '=' + str(val)
    r = requests.get(request_url)
    j = r.json()
    return j


def gmaps_get_next_page(api_key, next_page_token):
    """get next search engine results page page using search token, waiting until available"""
    r = requests.get(GMAPS_URL + '?pagetoken=' + next_page_token +
                        '&key=' + api_key)
    for i in range(10):
        j = r.json()
        if not j['results']: # wait for next page to be available
            time.sleep(5)
            continue
        else:
            return j


def gmaps_get_all_df(api_key, location, **kwargs):
    """return dataframe of all results using api_key, location, search kwargs"""
    # get first page
    j = gmaps_get_first_page(api_key, location, **kwargs)
    venues_df = pd.json_normalize(j['results'])

    # get pages while additional pages available
    while 'next_page_token' in j:
        next_page_token = j['next_page_token']
        time.sleep(5)
        j = gmaps_get_next_page(api_key, next_page_token)
        venues_df = venues_df.append(pd.json_normalize(j['results']))
        
    return venues_df


def gmaps_get_df(location_coords, keyword):

    # use either rankby or radius
    location = "%.7f,%.7f" % location_coords
    gmaps_df = gmaps_get_all_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
    if gmaps_df.empty:
        return None
    else:
        # gmaps_get_df(api_key, location, keyword=keyword, ltype=ltype, radius=RADIUS)
        gmaps_df = gmaps_df.loc[(gmaps_df['user_ratings_total'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
                           .sort_values(['rating', 'user_ratings_total'], ascending=False) \
                           .reset_index(drop=True)
        gmaps_df = gmaps_df[['name', 'vicinity', 'rating', 'user_ratings_total', 'geometry.location.lat', 'geometry.location.lng']]
        gmaps_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng']
        # drop trailing ", Brooklyn"
        gmaps_df['address'] = gmaps_df['address'].apply(lambda address: " ".join(address.split(',')[:-1]))
        gmaps_df['distance'] = gmaps_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        return gmaps_df


In [75]:
%%time
gmaps_df = gmaps_get_df(location_coords, keyword)
gmaps_df = gmaps_df.loc[(gmaps_df['nratings'] >= MIN_USER_RATINGS) & (gmaps_df['rating'] >= MIN_RATING)] \
        .sort_values(['rating', 'nratings'], ascending=False) \
        .reset_index(drop=True)
gmaps_df.to_pickle('gmaps_' + test_pickle_filename)
gmaps_df


CPU times: user 96.5 ms, sys: 5.9 ms, total: 102 ms
Wall time: 13.6 s


,name,address,rating,nratings,lat,lng,distance
0,Rice & Miso,134 Nevins St,4.7,73,40.684593,-73.983818,1.249951
1,San Blas Fine Mexican Cuisine,140 Montague St,4.7,47,40.694469,-73.993729,0.350772
2,Marco Polo,345 Court St,4.6,362,40.682223,-73.995793,1.039714
3,Hibino,333 Henry St,4.6,316,40.690148,-73.996342,0.177603
4,Saketumi,118 Montague St,4.6,208,40.694729,-73.994774,0.353686
5,Westville,81 Washington St,4.5,1039,40.702085,-73.989448,1.270588
6,French Louie,320 Atlantic Ave,4.5,705,40.688014,-73.988183,0.728030
7,Rucola,190 Dean St,4.5,674,40.685611,-73.985892,1.042501
8,Queen,84 Court St,4.5,447,40.691353,-73.991772,0.308502
9,Mooburger,240 Court St,4.5,281,40.685834,-73.994454,0.643327


In [76]:
# sort grid by clicking on header (can also click on filter button)
show_grid(gmaps_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [77]:
# plot on google map

markers = [(row.lat, row.lng) for row in gmaps_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in gmaps_df.itertuples()]
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Google rating</dt><dd>{rating}</dd>
<dt>Google reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**row) for i, row in gmaps_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=eval(location), zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [78]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
    
venues_map

## Yelp

In [79]:
def yelp_get_df(location_coords, keyword):
    lat, lng = location_coords
    response = yelp_api.search_query(categories=keyword, latitude=lat, longitude=lng, 
                                     radius=RADIUS, sort_by=rankby, limit=NRESULTS)

    yelp_df = pd.json_normalize(response['businesses'])
    if not yelp_df.empty:    
        yelp_df = yelp_df.loc[(yelp_df['review_count'] >= MIN_USER_RATINGS) & (yelp_df['rating'] >= MIN_RATING)] \
                         .sort_values(['rating', 'review_count'], ascending=False) \
                         .reset_index(drop=True)
        display_columns = ['name', 'location.address1', 'rating', 'review_count', 'coordinates.latitude', 'coordinates.longitude', 'url']
        yelp_df = yelp_df[display_columns]
        yelp_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        yelp_df['distance'] = yelp_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                              axis=1)
        
        if yelp_df.empty:
            return None
        else:
            return yelp_df
    else:
        return None

yelp_df = yelp_get_df(location_coords, keyword)
yelp_df

,name,address,rating,nratings,lat,lng,url,distance
0,Lillo Cucina Italiana,331 Henry St,5.0,138,40.690230,-73.996340,https://www.yelp.com/biz/lillo-cucina-italiana...,0.169412
1,Ella Crown Bakehouse,149 Atlantic Ave,5.0,21,40.690691,-73.995072,https://www.yelp.com/biz/ella-crown-bakehouse-...,0.102889
2,Yemen Cafe & Restaurant,176 Atlantic Ave,4.5,513,40.689917,-73.993656,https://www.yelp.com/biz/yemen-cafe-and-restau...,0.236909
3,Sahadi's,187 Atlantic Ave,4.5,380,40.690218,-73.993367,https://www.yelp.com/biz/sahadis-brooklyn?adju...,0.229615
4,Damascus Bread & Pastry Shop,195 Atlantic Ave,4.5,252,40.690050,-73.993070,https://www.yelp.com/biz/damascus-bread-and-pa...,0.260803
5,Boutros,185 Atlantic Ave,4.5,119,40.690148,-73.993545,https://www.yelp.com/biz/boutros-brooklyn?adju...,0.223999
6,Elsa,136 Atlantic Ave,4.5,106,40.690270,-73.995350,https://www.yelp.com/biz/elsa-brooklyn?adjust_...,0.145693
7,Espresso Me,88 Atlantic Ave,4.5,57,40.690833,-73.997073,https://www.yelp.com/biz/espresso-me-brooklyn-...,0.163364
8,Two For the Pot,200 Clinton St,4.5,36,40.690670,-73.994461,https://www.yelp.com/biz/two-for-the-pot-brook...,0.129099
9,Banhmigos,92 Court St,4.5,23,40.691110,-73.991840,https://www.yelp.com/biz/banhmigos-new-york-2?...,0.306221


In [80]:
yelp_df.to_pickle('yelp_' + test_pickle_filename)
show_grid(yelp_df)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [81]:
markers = [(row.lat, row.lng) for row in yelp_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in yelp_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Yelp rating</dt><dd>{rating}</dd>
<dt>Yelp reviews</dt><dd>{nratings}</dd>
</dl>
"""

marker_info = [info_box_template.format(**row) for i, row in yelp_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [82]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Foursquare

In [83]:
def foursquare_get_df(location_coords, keyword):
    location_str = "%.7f,%.7f" % location_coords
    client = Foursquare(client_id=foursquare_id, 
                                   client_secret=foursquare_secret, 
                                   redirect_uri='http://streeteye.com/')
    response = client.venues.search(params={'query': keyword, 'll': "%s" % location_str, 
                                            'radius': RADIUS, 'limit': NRESULTS})

    foursquare_array = []

    for i, venue in pd.json_normalize(response['venues']).iterrows():
        venue_id = venue['id']
        # query detailed venue info from foursquare
        venue_name = venue['name']
        venue_address = venue['location.address']
        # sometimes no URL
        try:
            venue_url = venue['delivery.url']
        except:
            venue_url = ''
        venue_lat = venue['location.lat']
        venue_lng = venue['location.lng']
        # default these to -1
        try:
            # get rating, nratings with another API call for venue details
            venue_details = client.venues(venue_id)['venue']
            venue_rating = venue_details['rating']
            venue_nratings = venue_details['ratingSignals']
        except FoursquareException as e:
            print("Foursquare exception", type(e), str(e))
        except Exception as e:
            continue
            # sometimes no rating ... probably not popular enough
            # print(type(e), str(e))
            # print(traceback.format_exc())
            # print("No rating for %s" % venue_name)

        foursquare_array.append([venue_name, venue_address, venue_rating, venue_nratings, venue_lat, venue_lng, venue_url])
            
    foursquare_df = pd.DataFrame(foursquare_array)
    
    if len(foursquare_df) and len(foursquare_df.columns):
        foursquare_df.columns = ['name', 'address', 'rating', 'nratings', 'lat', 'lng', 'url']
        foursquare_df = foursquare_df.loc[(foursquare_df['nratings'] >= MIN_USER_RATINGS) & (foursquare_df['rating'] >= MIN_RATING)] \
                                     .sort_values(['rating', 'nratings'], ascending=False) \
                                     .reset_index(drop=True)
        
        foursquare_df['distance'] = foursquare_df.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
                                                        axis=1)
        if foursquare_df.empty:
            return None
        else:
            return foursquare_df
    else:
        return None

foursquare_df = foursquare_get_df(location_coords, keyword)
foursquare_df

In [84]:
if foursquare_df is not None and not foursquare_df.empty:
    foursquare_df.to_pickle('foursquare_' + test_pickle_filename)
show_grid(foursquare_df)

TypeError: data_frame must be DataFrame or Series, not <class 'NoneType'>

In [ ]:
markers = [(row.lat, row.lng) for row in foursquare_df.itertuples()]
marker_hover = ["%s: %s (%s)" % (row.name, row.rating, row.nratings) for row in foursquare_df.itertuples()]

info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
<dt>Foursquare rating</dt><dd>{rating}</dd>
<dt>Foursquare reviews</dt><dd>{nratings}</dd>
</dl>
"""
marker_info = [info_box_template.format(**d_item) for i, d_item in foursquare_df.iterrows()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig

In [ ]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map

## Deduplicate and merge

In [85]:
# put everything in one big df
pd.set_option('display.max_rows', None)

gmaps_df_copy, yelp_df_copy, foursquare_df_copy = None, None, None

try:
    gmaps_df_copy = gmaps_df.copy()
    gmaps_df_copy['source'] = '0_gmaps'
except: 
    pass

try:
    yelp_df_copy = yelp_df.copy() 
    yelp_df_copy['source'] = '1_yelp'
except:
    pass

try:
    foursquare_df_copy = foursquare_df.copy()
    foursquare_df_copy['source'] = '2_foursquare'
except:
    pass


venues_df = pd.concat(list(filter(lambda df: df is not None, [gmaps_df_copy, yelp_df_copy, foursquare_df_copy]))).reset_index()
venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)

venues_df.sort_values('name')

,index,name,address,rating,nratings,lat,lng,distance,source,url,latlong
13,13,61 Local,61 Bergen St,4.4,581,40.687137,-73.990879,0.624641,0_gmaps,NaN,"(40.6871375, -73.99087879999999)"
14,14,AlMar,111 Front St,4.4,530,40.702664,-73.989174,1.338769,0_gmaps,NaN,"(40.7026637, -73.98917449999999)"
90,32,Atlantic Bagels Cafe,200 Clinton St,3.0,105,40.690671,-73.994460,0.129065,1_yelp,https://www.yelp.com/biz/atlantic-bagels-cafe-...,"(40.690671, -73.99446)"
54,54,Au Bon Pain,70 Myrtle Ave,3.8,154,40.693604,-73.986540,0.782624,0_gmaps,NaN,"(40.6936043, -73.9865401)"
67,9,Banhmigos,92 Court St,4.5,23,40.691110,-73.991840,0.306221,1_yelp,https://www.yelp.com/biz/banhmigos-new-york-2?...,"(40.69111, -73.99184)"
21,21,Bareburger,149 Court St,4.3,863,40.689198,-73.992383,0.368045,0_gmaps,NaN,"(40.6891984, -73.9923831)"
75,17,Beasts & Bottles,151 Atlantic Ave,4.0,173,40.690590,-73.995030,0.114650,1_yelp,https://www.yelp.com/biz/beasts-and-bottles-br...,"(40.69059, -73.99503)"
37,37,Bevacco,60 Henry St,4.2,251,40.699190,-73.992324,0.884316,0_gmaps,NaN,"(40.6991904, -73.992324)"
63,5,Boutros,185 Atlantic Ave,4.5,119,40.690148,-73.993545,0.223999,1_yelp,https://www.yelp.com/biz/boutros-brooklyn?adju...,"(40.6901481293837, -73.9935445412993)"
77,19,Brado,155 Atlantic Ave,4.0,137,40.690557,-73.995479,0.113885,1_yelp,https://www.yelp.com/biz/brado-brooklyn-3?adju...,"(40.6905570310882, -73.9954792872909)"


In [86]:
# manual fix 
# venues_df.loc[venues_df['address']=='212b Pacific St', 'address'] = '212 Pacific St'
# venues_df

In [87]:
# run dedupe algorithm using name, address as default texts, latlong as latlong
venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])
venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['name', 'address', ('latlong', 'LatLong')])


Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 89


In [88]:
# view clustering
venues_df['cluster'] = venues_df2['cluster id']
venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'distance', 'source']]
venues_df

,cluster,name,address,rating,nratings,lat,lng,distance,source
3,0,Hibino,333 Henry St,4.6,316,40.690148,-73.996342,0.17760290052479702,0_gmaps
69,0,Hibino,333 Henry St,4.0,400,40.69017,-73.99637,0.17649310902692844,1_yelp
8,1,Queen,84 Court St,4.5,447,40.69135259999999,-73.99177209999999,0.3085021511478388,0_gmaps
71,1,Queen Italian Restaurant,84 Court St,4.0,309,40.691356,-73.991905,0.2972761325611104,1_yelp
27,2,CAVA,345 Adams St,4.3,214,40.6925169,-73.988608,0.5842152408106424,0_gmaps
36,2,Potbelly Sandwich Shop,345 Adams St,4.2,269,40.6928562,-73.9883451,0.613677370574672,0_gmaps
28,3,Mitoushi Sushi,177 Atlantic Ave,4.3,142,40.6902609,-73.9938298,0.19830773518673966,0_gmaps
86,3,Mitoushi Sushi,177 Atlantic Ave,3.5,176,40.69034,-73.993783,0.19467779495964357,1_yelp
52,4,Popeyes Louisiana Kitchen,80 Court St,3.8,638,40.6914675,-73.9918296,0.30285871705786155,0_gmaps
92,4,Popeyes Louisiana Kitchen,80 Court St,2.5,60,40.69144,-73.99169,0.3147860998606376,1_yelp


In [89]:
# group by clusters
cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source']] \
    .first() \
    .reset_index()
cluster_df

,cluster,name,address,lat,lng,distance,source
0,0,Hibino,333 Henry St,40.690148,-73.996342,0.17760290052479702,0_gmaps
1,1,Queen,84 Court St,40.69135259999999,-73.99177209999999,0.3085021511478388,0_gmaps
2,2,CAVA,345 Adams St,40.6925169,-73.988608,0.5842152408106424,0_gmaps
3,3,Mitoushi Sushi,177 Atlantic Ave,40.6902609,-73.9938298,0.19830773518673966,0_gmaps
4,4,Popeyes Louisiana Kitchen,80 Court St,40.6914675,-73.9918296,0.30285871705786155,0_gmaps
5,5,Rice & Miso,134 Nevins St,40.68459259999999,-73.98381789999999,1.249951456086607,0_gmaps
6,6,San Blas Fine Mexican Cuisine,140 Montague St,40.6944695,-73.99372939999999,0.3507715987090645,0_gmaps
7,7,Marco Polo,345 Court St,40.682223,-73.99579299999999,1.0397136087137755,0_gmaps
8,8,Saketumi,118 Montague St,40.6947292,-73.9947737,0.35368566047102395,0_gmaps
9,9,Westville,81 Washington St,40.7020849,-73.98944809999999,1.2705882174373355,0_gmaps


In [90]:
# make markers on clusters
# add ratings on all rows

markers = [(float(row.lat), float(row.lng)) for row in cluster_df.itertuples()]
marker_hover = ["%s" % (row.name) for row in cluster_df.itertuples()]

# make a dict by cluster, initialize rating string to ''
marker_dict = {i: {'name': row['name'],
                   'address': row['address'],
                   'lat': row['lat'],
                   'lng': row['lng'],
                   'rate_str': '',
                  } for i, row in cluster_df.iterrows()}

# add all rating strings
for i, row in venues_df.iterrows():
    cluster = row['cluster']
    if row.source[2:] == 'gmaps':
        marker_dict[cluster]['rate_str'] += "<dt>Google rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'yelp':
        marker_dict[cluster]['rate_str'] += "<dt>Yelp rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    elif row.source[2:] == 'foursquare':
        marker_dict[cluster]['rate_str'] += "<dt>Foursquare rating</dt><dd>%s (%s reviews)</dd>\n" % (row.rating, row.nratings)
    
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
{rate_str}

</dl>
"""    

marker_info = [info_box_template.format(**d_item) for i, d_item in marker_dict.items()]

marker_layer = gmaps.marker_layer(markers, hover_text=marker_hover, info_box_content=marker_info)

figure_layout = {
    'width': '800px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px'
}

fig = gmaps.figure(layout=figure_layout, center=location_coords, zoom_level=14)
fig.add_layer(marker_layer)
fig


Figure(layout=FigureLayout(border='1px solid black', height='800px', padding='1px', width='800px'))

In [91]:
folium_markers = [(a[0], a[1], b) for a, b in zip(markers, marker_info)]

venues_map = folium.Map(location=[*location_coords], zoom_start=14)
for lat, lng, label in folium_markers:
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        color='blue',
        tooltip=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.5
    ).add_to(venues_map)
venues_map


In [92]:
venues_df

,cluster,name,address,rating,nratings,lat,lng,distance,source
3,0,Hibino,333 Henry St,4.6,316,40.690148,-73.996342,0.17760290052479702,0_gmaps
69,0,Hibino,333 Henry St,4.0,400,40.69017,-73.99637,0.17649310902692844,1_yelp
8,1,Queen,84 Court St,4.5,447,40.69135259999999,-73.99177209999999,0.3085021511478388,0_gmaps
71,1,Queen Italian Restaurant,84 Court St,4.0,309,40.691356,-73.991905,0.2972761325611104,1_yelp
27,2,CAVA,345 Adams St,4.3,214,40.6925169,-73.988608,0.5842152408106424,0_gmaps
36,2,Potbelly Sandwich Shop,345 Adams St,4.2,269,40.6928562,-73.9883451,0.613677370574672,0_gmaps
28,3,Mitoushi Sushi,177 Atlantic Ave,4.3,142,40.6902609,-73.9938298,0.19830773518673966,0_gmaps
86,3,Mitoushi Sushi,177 Atlantic Ave,3.5,176,40.69034,-73.993783,0.19467779495964357,1_yelp
52,4,Popeyes Louisiana Kitchen,80 Court St,3.8,638,40.6914675,-73.9918296,0.30285871705786155,0_gmaps
92,4,Popeyes Louisiana Kitchen,80 Court St,2.5,60,40.69144,-73.99169,0.3147860998606376,1_yelp


In [93]:
merge_df = cluster_df \
    .merge(venues_df.loc[venues_df['source']=='0_gmaps'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='1_yelp'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

merge_df = merge_df \
    .merge(venues_df.loc[venues_df['source']=='2_foursquare'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
    .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

merge_df 

/Users/drucev/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/extmath.py:765: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/drucev/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/extmath.py:706: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


,cluster,name,address,lat,lng,distance,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std
0,0,Hibino,333 Henry St,40.690148,-73.996342,0.17760290052479702,0_gmaps,4.6,316,1.423044,4.0,400,0.113083,NaN,NaN,NaN
1,1,Queen,84 Court St,40.69135259999999,-73.99177209999999,0.3085021511478388,0_gmaps,4.5,447,1.044436,4.0,309,0.113083,NaN,NaN,NaN
2,2,CAVA,345 Adams St,40.6925169,-73.988608,0.5842152408106424,0_gmaps,4.3,214,0.287220,NaN,NaN,NaN,NaN,NaN,NaN
3,2,CAVA,345 Adams St,40.6925169,-73.988608,0.5842152408106424,0_gmaps,4.2,269,-0.091388,NaN,NaN,NaN,NaN,NaN,NaN
4,3,Mitoushi Sushi,177 Atlantic Ave,40.6902609,-73.9938298,0.19830773518673966,0_gmaps,4.3,142,0.287220,3.5,176,-0.701113,NaN,NaN,NaN
5,4,Popeyes Louisiana Kitchen,80 Court St,40.6914675,-73.9918296,0.30285871705786155,0_gmaps,3.8,638,-1.605821,2.5,60,-2.329506,NaN,NaN,NaN
6,5,Rice & Miso,134 Nevins St,40.68459259999999,-73.98381789999999,1.249951456086607,0_gmaps,4.7,73,1.801653,NaN,NaN,NaN,NaN,NaN,NaN
7,6,San Blas Fine Mexican Cuisine,140 Montague St,40.6944695,-73.99372939999999,0.3507715987090645,0_gmaps,4.7,47,1.801653,NaN,NaN,NaN,NaN,NaN,NaN
8,7,Marco Polo,345 Court St,40.682223,-73.99579299999999,1.0397136087137755,0_gmaps,4.6,362,1.423044,NaN,NaN,NaN,NaN,NaN,NaN
9,8,Saketumi,118 Montague St,40.6947292,-73.9947737,0.35368566047102395,0_gmaps,4.6,208,1.423044,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
# simple average score
merge_df['meanrating'] = np.nanmean(merge_df[['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']], axis=1)
merge_df.sort_values('meanrating', ascending=False)[['name', 'address', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'meanrating']]


,name,address,gmaps_rating,yelp_rating,foursquare_rating,meanrating
6,Rice & Miso,134 Nevins St,4.7,NaN,NaN,1.801653
7,San Blas Fine Mexican Cuisine,140 Montague St,4.7,NaN,NaN,1.801653
58,Lillo Cucina Italiana,331 Henry St,NaN,5.0,NaN,1.741475
59,Ella Crown Bakehouse,149 Atlantic Ave,NaN,5.0,NaN,1.741475
8,Marco Polo,345 Court St,4.6,NaN,NaN,1.423044
9,Saketumi,118 Montague St,4.6,NaN,NaN,1.423044
15,Govinda's Vegetarian Lunch,305 Schermerhorn St,4.5,NaN,NaN,1.044436
11,French Louie,320 Atlantic Ave,4.5,NaN,NaN,1.044436
14,Fast and Fresh Burrito Deli,84 Hoyt St,4.5,NaN,NaN,1.044436
13,Mooburger,240 Court St,4.5,NaN,NaN,1.044436


In [95]:
# bayes score
rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
nratings_mean = np.mean(merge_df['nratings'])
rating_avg = np.nanmean(merge_df[rating_cols])
merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
print('mean number of ratings', nratings_mean)
print('average rating', rating_avg)
merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg

show_grid(merge_df.sort_values('bayes_score', ascending=False)[['name', 'address', 'distance', 'gmaps_rating', 'yelp_rating', 'foursquare_rating', 'nratings', 'bayes_score']])

mean number of ratings 1.0444444444444445
average rating 1.3842355158092378e-15


QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [96]:
def dedupe(dedupe_list):

    for i, source_df in enumerate(dedupe_list):
        source_df['source'] = i
    venues_df = pd.concat(dedupe_list).reset_index()
    venues_df['latlong'] = venues_df[['lat','lng']].apply(tuple, axis=1)
    venues_df['shortname'] = venues_df['name'].apply(lambda n: n[:25])

    # dedupe and assign cluster id
    venues_df2 = pandas_dedupe.dedupe_dataframe(venues_df, ['shortname', 'address', ('latlong', 'LatLong')])
    venues_df['cluster'] = venues_df2['cluster id']
    venues_df = venues_df.sort_values(['cluster', 'source'])[['cluster', 'name', 'address', 'rating', 'nratings', 'lat', 'lng', 'distance', 'source']]

    # group by clusters, uniquify name
    cluster_df = venues_df.groupby('cluster')[['name', 'address', 'lat', 'lng', 'distance', 'source']] \
                          .first() \
                          .reset_index()

    # merge ratings by source
    merge_df = cluster_df \
        .merge(venues_df.loc[venues_df['source']=='0'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'gmaps_rating', 'nratings': 'gmaps_nratings'})
    merge_df['gmaps_rating_std'] = StandardScaler().fit_transform(merge_df[['gmaps_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='1'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'yelp_rating', 'nratings': 'yelp_nratings'})
    merge_df['yelp_rating_std'] = StandardScaler().fit_transform(merge_df[['yelp_rating']])

    merge_df = merge_df \
        .merge(venues_df.loc[venues_df['source']=='2'][['cluster','rating', 'nratings']], on='cluster', how='outer') \
        .rename(columns={'rating': 'foursquare_rating', 'nratings': 'foursquare_nratings'})
    merge_df['foursquare_rating_std'] = StandardScaler().fit_transform(merge_df[['foursquare_rating']])

    # bayes score
    rating_cols = ['gmaps_rating_std', 'yelp_rating_std', 'foursquare_rating_std']
    merge_df['nratings'] = merge_df[rating_cols].count(axis=1)
    nratings_mean = np.mean(merge_df['nratings'])
    rating_avg = np.nanmean(merge_df[rating_cols])
    merge_df['w'] = merge_df['nratings']/(merge_df['nratings'] + nratings_mean)
    merge_df['R'] = np.mean(merge_df[rating_cols], axis=1)
    merge_df['bayes_score'] = merge_df['w'] * merge_df['R'] + (1 - merge_df['w']) * rating_avg
    merge_df = merge_df.sort_values('bayes_score', ascending=False)
    return merge_df

    
dedupe_list = list(filter(lambda df: df is not None, [gmaps_df, yelp_df, foursquare_df]))

dedupe_df = dedupe(dedupe_list)
dedupe_df

Importing data ...
Reading from dedupe_dataframe_learned_settings
Clustering...
# duplicate sets 89


/Users/drucev/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/extmath.py:765: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/Users/drucev/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/extmath.py:706: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = op(x, *args, **kwargs)


,cluster,name,address,lat,lng,distance,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std,nratings,w,R,bayes_score
6,5,Rice & Miso,134 Nevins St,40.68459259999999,-73.98381789999999,1.249951456086607,0,4.7,73,1.801653,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.801653,0.881243
7,6,San Blas Fine Mexican Cuisine,140 Montague St,40.6944695,-73.99372939999999,0.3507715987090645,0,4.7,47,1.801653,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.801653,0.881243
58,57,Lillo Cucina Italiana,331 Henry St,40.69023,-73.99634,0.16941248622763924,1,NaN,NaN,NaN,5.0,138,1.741475,NaN,NaN,NaN,1,0.489130,1.741475,0.851809
59,58,Ella Crown Bakehouse,149 Atlantic Ave,40.690691,-73.995072,0.10288887380250279,1,NaN,NaN,NaN,5.0,21,1.741475,NaN,NaN,NaN,1,0.489130,1.741475,0.851809
8,7,Marco Polo,345 Court St,40.682223,-73.99579299999999,1.0397136087137755,0,4.6,362,1.423044,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.423044,0.696054
9,8,Saketumi,118 Montague St,40.6947292,-73.9947737,0.35368566047102395,0,4.6,208,1.423044,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.423044,0.696054
15,14,Govinda's Vegetarian Lunch,305 Schermerhorn St,40.687646,-73.98216959999999,1.201442243379487,0,4.5,106,1.044436,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.044436,0.510866
12,11,Rucola,190 Dean St,40.6856111,-73.9858917,1.0425008691569033,0,4.5,674,1.044436,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.044436,0.510866
14,13,Fast and Fresh Burrito Deli,84 Hoyt St,40.6880866,-73.98697609999999,0.8116453841067722,0,4.5,188,1.044436,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.044436,0.510866
13,12,Mooburger,240 Court St,40.6858338,-73.99445399999999,0.6433269443232074,0,4.5,281,1.044436,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.044436,0.510866


In [97]:
dedupe_df.to_pickle(test_pickle_filename)
pd.read_pickle(test_pickle_filename)

,cluster,name,address,lat,lng,distance,source,gmaps_rating,gmaps_nratings,gmaps_rating_std,yelp_rating,yelp_nratings,yelp_rating_std,foursquare_rating,foursquare_nratings,foursquare_rating_std,nratings,w,R,bayes_score
6,5,Rice & Miso,134 Nevins St,40.68459259999999,-73.98381789999999,1.249951456086607,0,4.7,73,1.801653,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.801653,0.881243
7,6,San Blas Fine Mexican Cuisine,140 Montague St,40.6944695,-73.99372939999999,0.3507715987090645,0,4.7,47,1.801653,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.801653,0.881243
58,57,Lillo Cucina Italiana,331 Henry St,40.69023,-73.99634,0.16941248622763924,1,NaN,NaN,NaN,5.0,138,1.741475,NaN,NaN,NaN,1,0.489130,1.741475,0.851809
59,58,Ella Crown Bakehouse,149 Atlantic Ave,40.690691,-73.995072,0.10288887380250279,1,NaN,NaN,NaN,5.0,21,1.741475,NaN,NaN,NaN,1,0.489130,1.741475,0.851809
8,7,Marco Polo,345 Court St,40.682223,-73.99579299999999,1.0397136087137755,0,4.6,362,1.423044,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.423044,0.696054
9,8,Saketumi,118 Montague St,40.6947292,-73.9947737,0.35368566047102395,0,4.6,208,1.423044,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.423044,0.696054
15,14,Govinda's Vegetarian Lunch,305 Schermerhorn St,40.687646,-73.98216959999999,1.201442243379487,0,4.5,106,1.044436,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.044436,0.510866
12,11,Rucola,190 Dean St,40.6856111,-73.9858917,1.0425008691569033,0,4.5,674,1.044436,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.044436,0.510866
14,13,Fast and Fresh Burrito Deli,84 Hoyt St,40.6880866,-73.98697609999999,0.8116453841067722,0,4.5,188,1.044436,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.044436,0.510866
13,12,Mooburger,240 Court St,40.6858338,-73.99445399999999,0.6433269443232074,0,4.5,281,1.044436,NaN,NaN,NaN,NaN,NaN,NaN,1,0.489130,1.044436,0.510866


In [98]:
# locationdict = dict([('midtown', '40.7484, -73.9857'),
#                      ('downtown', '40.7077443,-74.0139089'),
#                      ('uppereastside', '40.7711473,-73.9661166'),
#                      ('upperwestside', '40.778794,-73.984257'),
#                      ('brooklynheights', '40.6915812,-73.9954095'), 
#                      ('grandarmyplaza', '40.671872,-73.972544'),
#                      ('bayridge', '40.6292633,-74.0309554'),
#                      ('williamsburg', '40.7144609,-73.9553373')])

# for k, l in product(['pizza', 'coffee', 'icecream'], 
#                     ['midtown','downtown','uppereastside','upperwestside','brooklynheights','grandarmyplaza','bayridge','williamsburg',]):
#     filename = "cache/%s_%s.pkl" % (k, l)
#     location_coords = eval(locationdict[l])
#     tempdf = pd.read_pickle(filename)
#     tempdf['distance'] = tempdf.apply(lambda row: distance((row['lat'], row['lng']), location_coords).km,
#                                       axis=1)
#     tempdf.to_pickle(filename)
#     print(filename)

In [99]:
# # file to retrain with train_dedupe2.py
# can train better if we add the keyword, not yet implemented
# tempdf = None
# for k, l, j in product(['pizza', 'coffee', 'icecream'], 
#                     ['midtown','downtown','uppereastside','upperwestside','brooklynheights','grandarmyplaza','bayridge','williamsburg',],
#                     ['gmaps', 'yelp', 'foursquare']):
#     filename = "cache/%s_%s_%s.pkl" % (j, k, l)
#     try:
#         if tempdf is None:
#             tempdf = pd.read_pickle(filename)
#             tempdf['keyword']=k
#             print(filename)
#         else:
#             tempdf2 = pd.read_pickle(filename)
#             tempdf2['keyword']=k
#             tempdf = pd.concat([tempdf, tempdf2])
#             print(filename)
#     except:
#         print("missing ", filename)